In [1]:
using GLMakie

GLMakie.activate!()

fig1 = Figure(size = (1000, 900))
ax = LScene(fig1[1, 1], show_axis=false; scenekw = (show_axis = false,))


f(X,Y,z) = begin
    w_x, w_y = 30., 20.;
    a = 0.;
    R = sqrt(a^2 + w_y^2);

    indicator(X, Y) = (abs.((X .- (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y)) +
                      (abs.(X) .< w_x) .* (abs.(Y) .< w_y) +
                      (abs.((X .+ (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y));

    return ((z < 1500.) ?
                5.e-3exp.(-((sqrt.(X .^ 2 + Y .^ 2) .- 8.) .^ 2) / 5. ^ 2) :
                2e-2 * indicator(X, Y)
                # 2e-1 * (abs.(X) .< 40.) .* (abs.(Y) .< 25.)
            );
end

r = LinRange(-100, 100, 200)
zs = LinRange(0, 3000, 200)
nVol = [f.(X,Y,z) for X in r, z in zs, Y in r]

nMin, nMax = minimum(nVol), maximum(nVol)
isovalue = .7nMax;
isorange = .3 * (nMax - nMin);

c = volume!(ax, (-100,+100),(0,+600),(-100,+100), nVol; alpha = .6, colormap = Reverse(:bone), algorithm = :iso, isovalue = isovalue, isorange = isorange, transparency = true)
Colorbar(fig1[1, 2], c, label = "Δn", height = 800, width = 10)

display(fig1)
save("guides-3d-iso.png", fig1);

<div align="center">

[![isosurface-plot](./guides-3d-iso.png)](#---)

</div>

In [3]:
using GLMakie

GLMakie.activate!()

fig1 = Figure(size = (1000, 900))
scene = LScene(fig1[1, 1], show_axis=false; scenekw = (show_axis = false,))

f(X,Y,z) = begin
    w_x, w_y = 30., 20.;
    a = 0.;
    R = sqrt(a^2 + w_y^2);

    indicator(X, Y) = (abs.((X .- (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y)) +
                      (abs.(X) .< w_x) .* (abs.(Y) .< w_y) +
                      (abs.((X .+ (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y));

    return ((z < 1500.) ?
                5.e-3exp.(-((sqrt.(X .^ 2 + Y .^ 2) .- 8.) .^ 2) / 5. ^ 2) :
                2e-2 * indicator(X, Y)
                # 2e-1 * (abs.(X) .< 40.) .* (abs.(Y) .< 25.)
            );
end

r = LinRange(-100, 100, 200)
zs = LinRange(0, 3000, 200)
nVol = [f.(X,Y,z) for X in r, z in zs, Y in r]

x = to_colormap(Reverse(:bone))
N = length(x)

colormap = [(x[i].r, x[i].g, x[i].b, (i/N)^.8) for i in 1:1:N]

c = volume!(scene, (-100,+100),(0,+600),(-100,+100), nVol; colormap = colormap, algorithm = :absorption, transparency = false)
Colorbar(fig1[1, 2], c, label = "Δn", height = 800, width = 10)


display(fig1)
save("guides-3d-absorption.png", fig1);


<div align="center">

[![absorption-plot](./guides-3d-absorption.png)](#---)

</div>

In [4]:
using GLMakie
using FFTW

fig = Figure(size = (1650, 500));

z = 0.;

ax = Dict(
    "Intensity" => Axis(fig[1, 1], title = "Intensity (z = $(z) μm)"),
    "Phase" => Axis(fig[1, 3], title = "Phase (z = $(z) μm)"),
    "Guides" => Axis(fig[1, 5], title = "Guides (z = $(z) μm)")
);

x = LinRange(-100., +100., 500);
y = LinRange(-100., +100., 500);

Field(x, y) = begin
    w_0 = 6.;

    X, Y = [coord for coord in x, _ in y], [coord for _ in x, coord in y];
    
    return @. exp(-(X^2 + Y^2) / (2 * w_0^2)) / (sqrt(2pi) * w_0) * (X + 1im * Y);
end

U = Field(x, y);

V(x, y, z) = begin
    
    X, Y = [coord for coord in x, _ in y], [coord for _ in x, coord in y];

    w_x, w_y = 30., 20.;
    a = 0.;
    R = sqrt(a^2 + w_y^2);

    indicator(X, Y) = (abs.((X .- (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y)) +
                      (abs.(X) .< w_x) .* (abs.(Y) .< w_y) +
                      (abs.((X .+ (w_x + a)).^2 + Y.^2) .< R ^ 2) .* (1. .- (abs.(X) .< w_x) .* (abs.(Y) .< w_y));

    return 1.45 .+ ((z < 1500.) ?
                5.e-3exp.(-((sqrt.(X .^ 2 + Y .^ 2) .- 8.) .^ 2) / 5. ^ 2) :
                2e-2 * indicator(X, Y)
                # 2e-1 * (abs.(X) .< 40.) .* (abs.(Y) .< 25.)
            );
end

ABC(x,y) = begin
    w, L = 20., 100.;
    X, Y = [coord for coord in x, _ in y], [coord for _ in x, coord in y];
    return ((sqrt.(X.^2 + Y.^2) .> L - w) .* ((sqrt.(X.^2 + Y.^2) .- (L - w)) / w) .^2);
end

hm = Dict(
    "Intensity" => heatmap!(ax["Intensity"], x, y,abs.(U), colormap = :inferno),
    "Phase" => heatmap!(ax["Phase"], x, y, angle.(U), colormap = :bone),
    "Guides" => heatmap!(ax["Guides"], x, y, V(x,y,0), colormap = Reverse(:grays))
);

cbar = Dict(
    "Intensity" => Colorbar(fig[1, 2], hm["Intensity"]),
    "Phase" => Colorbar(fig[1, 4], hm["Phase"]),
    "Guides" => Colorbar(fig[1, 6], hm["Guides"])
);

dz = 20.;

BC = ABC(x,y);

Diffract(U, dz) = begin
    dx, dy = step(x), step(y);
    Nx, Ny = length(x), length(y);

    n_0 = 1.45;
    k = 2pi * n_0 / 1050.e-3;

    Lap = [(2pi * ni_x)^2 + (2pi * ni_y)^2 for ni_x in fftfreq(Nx, 1/dx), ni_y in fftfreq(Ny, 1/dy)];

    return ifft(exp.(+1im * Lap * dz / 2k) .* fft(U));
end

for z in 0.:dz:6700.
    potential = V(x, y, z);
    U = Diffract(U, dz) .* exp.((-1im * potential  - BC) * dz);
    
    hm["Intensity"][3][] = abs.(U) .^ 2;
    hm["Phase"][3][] = angle.(U);
    hm["Guides"][3][] = potential;
    
    ax["Intensity"].title[] = "Intensity (z = $(z) μm)";
    ax["Phase"].title[] = "Phase (z = $(z) μm)";
    ax["Guides"].title[] = "Guides (z = $(z) μm)";

    display(fig);
    sleep(0.1);
end